# Poker Probability and Statistics

For several years, beginning in 2010 I made a living playing online poker professionally. Data Science was a natural progression for me as it requires a similar skill-set as earning a profit at online poker.  I wrote a blog about what data analysis has in common with poker, and I mentioned that Each time a poker hand is played at an online poker site, a hand history is generated that explains everything that each player did during the hand. I used software called Hold’em Manager (think Tableau for poker), which downloads each of these hand histories in real time to a PostgreSQL database so you can keep track of your opponent’s tendencies. 

Before we get started, a little background info is in order.  The game is No Limit Texas Hold’em, and if you’ve never played you can click here for a concise but detailed explanation.  In the business world, Data Science is used to make predictions and optimize decisions by creating machine learning models.  In online poker, the decision that needs to be made is whether to bet, call, or fold, but you aren’t allowed to use software to make that decision for you.  At most online poker sites, that is where the line is drawn in the rules.  This means that the model that must be trained is your brain, and the training is done away from the table with an endless stream of equity calculations.  

Anytime I ran into a situation while playing that confused me, I would mark the hand for review later. After my poker playing session was done, I'd go back through the hands that I'd marked for review and break them down mathematically so I'd have a better idea of what to do in each situation the next time it arose.  I picked out five hands from my poker career.  Using the statistics on my opponents that I had available at the time, I’ll explain my thought process.  Then you can analyze the hand using Python to determine which of your options offers the highest expected value.

**Note**: *When reviewing poker hands, it is common to refer to our opponent as the "Villain" and ourselves as the "Hero."*

## Hand 1 Overview:

![Hand 1 Overview Image here](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand1Overview_zpssezaxkhv.png)

- Hero bet 0.85 (Hero has 20.95 remaining);
- Opponent (Villain) raises to 2.50 (Villain has 30.35 remaining);
 - In poker terminology, this is called a 3bet. The small blind and big blind make the first bet, and the Hero raised them which was the second bet.
- There is currently 3.70 Total in the Pot.

Calling is not a good option for reasons that are beyond the scope of this blog post. The Hero must decide between raising with the plan of going All-In (Betting all remaining chips) or folding. Folding costs nothing so you will analyze the expected value of going all in. In this situation, I’d make a small raise to induce my opponent to All-in bluff, but we need to do the calculation as if I’m going All-in since that is the plan, so;

- Hero risks 20.95 if All-in and loses;
 - Going ‘All-in’ means to put all your money in the pot.  The Hero has 20.95 remaining, so going All-in risks 20.95.
- Hero wins 23 if All-in and wins;
 - There is currently 3.70 in the pot.  Hero bet 0.85 and the Villain raised to 2.50.  Hero must add 1.65 of remaining 20.95 to match Villain’s raise, leaving Hero with 19.30.  This means the Hero can win an additional 19.30 on top of the 3.70 already in the pot for a total of 23.00.
- Hero wins 3.70 if Villain folds.

In [1]:
Hand1_AllIn_Loses = -20.95
Hand1_AllIn_Winnings = 23
Hand1_Fold_Winnings = 3.7

### Hand 1 Relevant Statistics:

Poker is a game of deductive reasoning based on incomplete information.  Here is the information you have on this opponent:

![Stats1](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand1Stats_zpsoiap7ejk.png) ![Stats2](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand1Stats1_zpsgzqgflpg.png)

1. The Villian is in the Button position which is the first position to the right of the small and big blinds.  Overall, from this position, villian 3bets 7.4% (27 trials);
2. Hero is in the Cut-Off position, which is the first position to the right of the Button.  Overall, vs. the Cut-Off, villian 3bets 12.5% (16 trials);
3. When Villian is in the Button vs. a pre-flop raise from the Cut-Off, villian 3bets 25% (4 trials);
4. When Villian 3bets pre-flop and faces a raise, he folds 50% of the time (2 trials).

### Hand 1 Assumptions:

Based on the above statistics, I’m going to make the following assumptions which are educated guesses;

- Villian raises to 2.50 with about (~) 13-15% of the range of possible starting hands;
- Villian folds to a re-raise ~ 25% of the time and goes ‘All-in’ ~75% of the time;
- Villian re-raises ‘All-In’ with a ~10% range, which looks like this: ![9.7% range](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand1range_zpsu4o5h9zs.png)



- The hands highlighted in yellow represent the Villian's range, which consists of 128 of the 1326 possible combinations of starting hands (9.7%);
- If you’re wondering why A5s and A2s are in the range, those represent Villian’s bluff hands.



### Hand 1 Analysis:

Now that I have Villian’s range, I can plug the Hero's hand (10h10s) and the Villian’s range into an [equity calculator](http://www.acepokersolutions.com/Poker-equity-calculator/).  The equity calculator simulates 10h10s vs. Villian’s range thousands of times and determines that the Hero wins ~53.77% of the time.   

Now you can create variables for `Fold_Percent` and `Equity`

In [2]:
Hand1_Fold_Percent = .25
Hand1_Equity = .538

Now it’s a simple calculation.  You need to build a function that represents the following equation:

- $FoldEV = (FoldPercent * FoldWinnings)$

- $AllinEV = (1 - FoldPercent) * ((AllInWinnings * Equity) + (AllInLoses * (1 - Equity)))$

- $AllinExpectedValue = FoldEV + AllinEV$

In [3]:
def AllinExpectedValue(AllIn_Loses, AllIn_Winnings, Fold_Winnings, Fold_Percent, Equity):
    FoldEV = (Fold_Percent * Fold_Winnings)
    AllinEV = (1 - Fold_Percent) * ((AllIn_Winnings * Equity) + (AllIn_Loses * (1 - Equity)))
    AllinExpectedValue = FoldEV + AllinEV
    if AllinExpectedValue > 0:
        return 'Raise!', AllinExpectedValue
    else:
        return 'Fold!', AllinExpectedValue

Now plug the Hand 1 variables into the `AllinExpectedValue` function and print your results.

In [4]:
Hand1_Decision, Hand1_EV = AllinExpectedValue(Hand1_AllIn_Loses, Hand1_AllIn_Winnings, Hand1_Fold_Winnings, Hand1_Fold_Percent, Hand1_Equity)
print(Hand1_Decision)
print(Hand1_EV)

Raise!
2.9463250000000016


### Hand 1 Results

![Hand 1 Results Image here](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand1Results_zpsg21gqbkq.png)

**Hero wins the hand with a set (three of a kind) of Tens beating a pair of Kings.**

## Hand 2 Overview:

![Hand2 Overview](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand2Overview_zpsns6q7ysn.png)

- Hero bet 1.10 and has 28.58 remaining.
- Villain raises to 3.80 and has 10.66 remaining.
- There is currently 5.40 in the pot.

This is a similar situation as the last hand.  Again, calling is not a good option.  Folding costs nothing, so we need to determine if raising All-in has a positive expected value.

- Hero Risks 13.36 if All-in;
	- Villain has 10.66 remaining, and has raised 2.70;
- Hero wins 16.06 if All-in and wins hand;
	- Villain has 10.66 remaining, and there is currently 5.40 in pot;
- Hero wins 5.40 if All-in and Villain folds.

In [5]:
Hand2_AllIn_Loses = -13.36
Hand2_AllIn_Winnings = 16.06
Hand2_Fold_Winnings = 5.40

### Hand 2 Relevant Statistics:

![Hand2 Stats1](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand2Stats1_zpsn7emfe3p.png)
![Hand2 Stats2](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand2Stats2_zpsgvdea8b5.png)
![Hand2 Stats3](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand2Stats3_zpsl2wlvx0k.png)

1. The Villain is in the Small-Blind position.  From this position, Villain 3bets 7.2% (100+ trials);
	- 100+ trials = If there is no indication of the number of trials next to the relevant statistic;
2. The Hero is in the Cut-Off position, and vs. the Cut-Off, Villain 3bets 7.9% (89 trails);
3. The Villain folds to a 4bet 25% (8 trials);
4. The Villain 3bets 4% overall (100+ trails), but vs. the Hero only 1% (98 trials);
	- This is a huge red flag, and indicates the Villain has a very strong hand;
5. The Villain 3bets out of position vs. the Cut-Off 9.8% (51 trials);
	- Out of position means the Villain will be first to act in the later betting rounds, and Hero will be second to act.
6. The Villain folds to a 4bet when out of position 0% (5 trials);
	- Another huge red flag.

### Hand 2 Assumptions:

Based on the above statistics, I’m going to make the following assumptions;

- Vs. most opponents, the Villain 3bets ~8-10% of the time, but appears to 3bet 4 times as often vs. other opponents than the Hero.  This puts the Villain’s range at ~3%.
- The Villain has yet to fold to a 4bet in this situation, so at most the Villain is folding 10% of the time.
- The Villain calls with a ~2.7% rang, which looks like this: 

![Hand2 Range](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand2range_zpsezqjyylu.png)

- The hands in yellow represent the Villain’s assumed range, which consists of 34 of the 1326 possible combinations of starting hands (2.6%).

### Hand 2 Analysis:

Once again, I’ll use an equity calculator to determine the Hero’s equity.  In this case, the Hero’s hand has 35.84% equity vs. the Villain’s range.

Now you can create variables for `EstFoldPercent` and `Equity`. 


In [6]:
Hand2_Fold_Percent = .10
Hand2_Equity = .3584

Now plug the variables for this hand into the `AllinExpectedValue` function, and print the results.

In [7]:
Hand2_Decision, Hand2_EV = AllinExpectedValue(Hand2_AllIn_Loses, Hand2_AllIn_Winnings, Hand2_Fold_Winnings, Hand2_Fold_Percent, Hand2_Equity)
print(Hand2_Decision)
print(Hand2_EV)

Fold!
-1.994284799999999


### Hand 2 Results:

![Hand2 Results](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand2Results_zpsm16ctevc.png)

**The Hero wins the hand with a set of eights vs. two pairs, Aces and Kings.  This was a lucky result from a decision that would lose money in the long run.  The correct play here is to fold.**

## Hand 3 Overview:

![Hand3 Overview](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand3%20Overview_zpsxauhcafc.png)

- This hand is on the flop, and the Hero is first to act and checks (Hero has 25.08);
- The Villain bets 1.06 (Villain has 25.37 remaining);
- There is currently 2.66 in the pot.

Once again, calling is not a good option, and folding costs nothing.  The Hero must decide if raising in this situation, known as a Semi-Bluff, has a positive expected value.  A Semi-Bluff is when you raise with a drawing hand, in this case a flush draw.  The Hero does not have a made hand, currently only holding an Ace high, but has four Hearts and only needs one more for a flush which is a strong hand. Similarly, to the first hand, the best line of play when raising is to raise small and give the Villain the opportunity to bluff himself.  If the Hero were to just go all in, the Villain would most likely fold a hand like King of Hearts/Jack of Hearts.  

- Hero risks 25.08 if All-in;
- Hero wins 26.68 if All-in and wins hand;
	- The Villain bet 1.06, so the Hero must match that amount from the remaining 25.08, leaving the Hero with 24.02.  This means the Hero can win an additional 24.02 on top of the 2.66 currently in the pot for a total of 26.68;
- Hero wins 2.66 if Villain folds.


In [8]:
Hand3_AllIn_Loses = -25.08
Hand3_AllIn_Winnings = 26.68
Hand3_Fold_Winnings = 2.66

### Hand 3 Relevant Statistics:

![Hand3Stats1](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand3Stats1_zpsj39ohmy2.png)
![Hand3Stats2](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand3Stats2_zps5geq8hhb.png)
![Hand3Stats3](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand3Stats3_zpsz7ur6api.png)


1. Preflop, the Villain raised and from Middle Position the Villain raises 9% (100+ trails);
2. On the flop the Hero checks, and the Villain (who raised preflop) bets again.  This is called a continuation bet (Cbet).  The Villain Cbets 100% (12 trials);
3. The Hero is Check-Raising the Villain and when facing a Check-Raise, the Villain folds 67% (3 trials);


### Hand 3 Assumptions:

- Opponents preflop range is ~9-10%, which looks like this:

![Hand3 PreFlop Range](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand3PreflopRange_zpsz58cmukk.png)

- The hands highlighted in yellow represent the Villain’s range, which consists of 128 of the 1326 possible combinations of starting hands (9.7%);
- After Cbeting 100% of the time, the Villain folds ~67% of the time to the Hero’s Check-Raise, leaving ~35-45 combinations in the Villains range.  The Villains range looks like this:

![Hand3 Flop Range](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand3AllinRange_zpsovz2agzz.png)

- This range consists of 35 combinations, and make up the strongest hands from the Villain's pre-flop range.
- Some of the hands are different shades of yellow.  There are a couple reasons for this;
    1. Card removal - Pocket Aces (AA) normally has six combinations (Diamonds/Clubs, Hearts/Clubs, Spades/Clubs, Hearts/Diamonds, Spades/Diamonds, Spades/Hearts), but the Hero has the Ace of Hearts.  The Villain could only have three combinations of Pocket Aces (Diamonds/Clubs, Spades/Clubs, Spades/Diamonds).
    2. Flush Draws - There are four combinations of King/Jack suited (Diamonds, Hearts, Spades, and Clubs), but the Villain would only go All-in with one of those combos (Hearts).


### Hand 3 Analysis:

Once again, I plug the Hero’s hand and the Villain’s range into the equity calculator, this time also entering the 3 flop cards.  The equity calculator indicates that the Hero wins in this situation 43.58%.

Now you can create your variables for `EstFoldPercent` and `Equity`.


In [9]:
Hand3_Fold_Percent = .67
Hand3_Equity = .4358

Now plug the Hand 3 variables into the `AllinExpectedValue` function and print the results.

In [10]:
Hand3_Decision, Hand3_EV = AllinExpectedValue(Hand3_AllIn_Loses, Hand3_AllIn_Winnings, Hand3_Fold_Winnings, Hand3_Fold_Percent, Hand3_Equity)
print(Hand3_Decision)
print(Hand3_EV)

Raise!
0.9496126400000008


### Hand 3 Results:

![Hand3 Results](http://i1285.photobucket.com/albums/a584/daniel_poston1/Hand3Results_zpsf7srwfja.png)

**The Hero wins with and Ace high flush, beating the Villains pair of Tens.**